In [ ]:
import pandas as pd
import numpy as np
import string
import re
import nltk
import gensim
nltk.download('cmudict')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('word_tokenize')
nltk.download('WordNetLemmatizer')
nltk.download('punkt_tab')

# nltk.data.path.insert(0, "/home/romaric/code/Romaric1209/GAIDI/notebooks/roma_NTLK_Data_Cache")
# nltk.data.path.append("/home/romaric/code/Romaric1209/GAIDI/notebooks/roma_NTLK_Data_Cache")
# nltk.download('cmudict', download_dir="/home/romaric/code/Romaric1209/GAIDI/notebooks/roma_NTLK_Data_Cache")
# nltk.download('punkt', download_dir="/home/romaric/code/Romaric1209/GAIDI/notebooks/roma_NTLK_Data_Cache")
# nltk.download('stopwords', download_dir="/home/romaric/code/Romaric1209/GAIDI/notebooks/roma_NTLK_Data_Cache")
# nltk.download('wordnet', download_dir="/home/romaric/code/Romaric1209/GAIDI/notebooks/roma_NTLK_Data_Cache")
from nltk.corpus import cmudict, stopwords, wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from collections import Counter
from textblob import TextBlob
from gensim.models import LsiModel
from gensim import corpora
from gensim.models.coherencemodel import CoherenceModel
import textstat

# Functions!

In [ ]:

cmu_dict = cmudict.dict()  # This should load from the cache
print(cmu_dict["hello"])

In [ ]:
def word_count(text):
    if not isinstance(text, str):  # Convert to string if it's not
       text = str(text)
    return len(text.split())

In [ ]:
def basic_cleaning(text):
    if not isinstance(text, str):  # Convert to string if it's not
       text = str(text)
    # Remove whitespace
    prepoc_text = text.strip()
    # Lowercasing
    prepoc_text = prepoc_text.lower()
    # remove digits
    prepoc_text = "".join(char for char in prepoc_text if not char.isdigit())
    # remove punctuation
    for punctuation in string.punctuation:
        prepoc_text = prepoc_text.replace(punctuation," ")
    # remove regex
    prepoc_text = re.sub('<[^<]+?',"",prepoc_text)

    return prepoc_text

In [ ]:
def cons_density(text):

    consonnant = sum(1 for char in text if char.isalpha() and char not in "aeiou")
    vowel = sum(1 for char in text if char.isalpha() and char in "aeiou")
    total_letters = vowel + consonnant
    return round((consonnant/(vowel + consonnant)),3) if total_letters > 0 else 0

In [ ]:
cmu_dict = cmudict.dict()

def get_word_stress(word):
    if word in cmu_dict:
        return sum(int(char) for syllable in cmu_dict[word][0] for char in syllable if char.isdigit())
    return 0

def get_sentence_stress(sentence):
    words = sentence.split()
    stress_values = [get_word_stress(word) for word in words]
    return sum(stress_values)

In [ ]:
def redundance(text):
    # give a redundance score, considering the lenght of each text, if a lemmatized words appears more than three times the mean, it is considered redundant.

    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    clean_tokens = [w for w in tokens if w not in stop_words]

    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(w) for w in clean_tokens]

    word_counts = Counter(lemmatized_tokens)
    mean_freq = sum(word_counts.values()) / len(word_counts) if len(word_counts)!= 0 else 0

    if mean_freq != 0:
        score = sum(1 for word, count in word_counts.items() if count > 2.5 * mean_freq)
    else:
        score = 0

    return score

In [ ]:
def sentiment_polarity(text):
    sent_pol = TextBlob(text).sentiment.polarity
    return abs(round(sent_pol,3))

In [ ]:
def word_choice(text):
    common_ai_words =["commendable",'transhumanist', 'meticulous', 'elevate','hello', 'tapestry','leverage',
                  'journey', 'headache','resonate','testament','explore', 'binary','delve',
                  'enrich', 'seamless','multifaceted', 'sorry','foster', 'convey', 'beacon',
                  'interplay', 'oh', 'navigate','form','adhere','cannot', 'landscape','remember',
                  'paramount', 'comprehensive', 'placeholder','grammar','real','summary','symphony',
                  'furthermore','relationship','ultimately','profound','art','supercharge','evolve',
                  'beyoud','reimagine','vibrant', 'robust','pivotal','certainly','quinoa','orchestrate','align',
                  'diverse','recommend','annals','note','employ','bustling','indeed','digital','enigma', 'outfit',
                  'indelible','refrain','culture','treat','emerge','meticulous','esteemed','weight','whimsical','bespoke',
                  'highlight','antagonist','unlock','key','breakdown','tailor','misinformation','treasure','paradigm','captivate',
                  'song','underscore','calculate','especially','climate','hedging','inclusive','exercise','ai','embrace',
                  'level','nuance','career','dynamic','accent','ethos','cheap','firstly','online','goodbye'
                  ]
    text = text.translate(str.maketrans('', '', string.punctuation)).lower()
    word_count = 0
    for word in text.split():
        if word in common_ai_words:
            word_count += 1

    return word_count

In [ ]:
def coherence(text):
    # uses gensim to measure coherence, use the lsi model(latent semantic indexing, coherence c_v because we provide the text)
    tokens = word_tokenize(text)
    if not tokens:
        coherence_score = 0
    else:
        dictionary = corpora.Dictionary([tokens])
        corpus_gensim = [dictionary.doc2bow(tokens)]
        lsa_model = LsiModel(corpus_gensim, id2word=dictionary)

        coherence_model = CoherenceModel(
            model=lsa_model,
            texts=[tokens],
            dictionary=dictionary,
            coherence='c_v'
        )
        coherence_score = coherence_model.get_coherence()
    return coherence_score

In [ ]:
def reading_ease(text):
    reading_ease= textstat.flesch_reading_ease(text)
    return reading_ease


def gunning_fog(text):
    gunning_fog = textstat.gunning_fog(text)
    return gunning_fog

In [ ]:
text = "Hello is the cat cat cat name, it is commendable. The cat eat the cat fish that was in the bowl of the cat, the cat is a bad cat!"
print(f'word count :{word_count(text)}')
print(f'cleaned :{basic_cleaning(text)}')
print(f'consonnance density :{cons_density(text)}')
print(f'stress value :{get_sentence_stress(text)}')
print(f'redundance :{redundance(text)}')
print(f'sentiment :{sentiment_polarity(text)}')
print(f'unusual word count :{word_choice(text)}')
print(f'coherence :{coherence(text)}')
print(f'reading ease :{reading_ease(text)}')
print(f'gunning fog :{gunning_fog(text)}')

# Create Pipeline
We want to add columns, not transform them ==> no ColumnTransformer <br>
Function transformer?<br>
But firt we need to get our preprocessed data...

In [ ]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import MinMaxScaler, FunctionTransformer

In [ ]:
import pandas as pd
data_load = pd.read_csv("/home/romaric/code/Romaric1209/GAIDI/data/1k_sampled_dataset.csv")
data = data_load.copy()
data.head()

In [ ]:
data["AI_gen"] = data["source"].apply(lambda x: 0 if x == "Human" else 1)

In [ ]:
X=pd.DataFrame(data["text"])
y=data["AI_gen"]

## Test No parallelism

In [ ]:
class InputHandler(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if isinstance(X, str):
            X = [X]
        if isinstance(X, list):
            X = pd.DataFrame({"text": X})
        elif isinstance(X, pd.DataFrame):
            if "text" not in X.columns:
                raise ValueError("Input DataFrame must have a 'text' column")
        else:
            X = pd.DataFrame({"text": list(X)})
        return X

class HowManyWords(BaseEstimator, TransformerMixin):
    def fit(self,X,y=None):
        return self

    def get_feature_names_out(self, input_features=None):
        return ["word_count"]

    def transform(self, X):
        if isinstance(X, pd.DataFrame):
            X = X["text"]
        word_c = X.apply(word_count)
        return pd.DataFrame({"word_count": word_c})

class TextPreprocessor(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def get_feature_names_out(self, input_features=None):
        return ["preprocessed"]

    def transform(self, X):
        if isinstance(X, pd.DataFrame):
            X = X["text"]
        cleaned = X.apply(basic_cleaning)
        return pd.DataFrame({"preprocessed": cleaned})

class ConsDensity(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def get_feature_names_out(self, input_features=None):
        return ["cons_density"]

    def transform(self, X):
        return X["preprocessed"].apply(cons_density).values.reshape(-1, 1)

class Stress(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def get_feature_names_out(self, input_features=None):
        return ["stress_value"]

    def transform(self, X):
        return X["preprocessed"].apply(get_sentence_stress).values.reshape(-1, 1)

class Sentiment(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def get_feature_names_out(self, input_features=None):
        return ["sentiment_score"]

    def transform(self, X):
        return X["preprocessed"].apply(sentiment_polarity).values.reshape(-1, 1)

class Redundance(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def get_feature_names_out(self, input_features=None):
        return ["redundance"]

    def transform(self, X):
        return X["preprocessed"].apply(redundance).values.reshape(-1, 1)

class UnusualWord(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def get_feature_names_out(self, input_features=None):
        return ["unusual_words"]

    def transform(self, X):
        return X["preprocessed"].apply(word_choice).values.reshape(-1, 1)

class Coherence(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def get_feature_names_out(self, input_features=None):
        return ["coherence"]

    def transform(self, X):
        return X["preprocessed"].apply(coherence).values.reshape(-1, 1)

class ReadingEase(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def get_feature_names_out(self, input_features=None):
        return ["reading_ease"]

    def transform(self, X):
        return X["text"].apply(reading_ease).values.reshape(-1, 1)

class GunningFog(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def get_feature_names_out(self, input_features=None):
        return ["gunning_fog"]

    def transform(self, X):
        return X["text"].apply(gunning_fog).values.reshape(-1, 1)


In [ ]:
log_scaler = FunctionTransformer(lambda x: np.log1p(x), validate=True)

pipeline = Pipeline([
    ("input_handler", InputHandler()),
    ("union", FeatureUnion([
        ("preprocessed_features", Pipeline([
            ("preprocessor", TextPreprocessor()),
            ("features", FeatureUnion([
                ("cons_density", ConsDensity()),
                ("stress_value", Pipeline([
                    ("extract", Stress()),
                    ("scaler", MinMaxScaler())
                ])),
                ("sentiment_score", Sentiment()),
                ("redundance", Pipeline([
                    ("extract", Redundance()),
                    ("log_scaling", log_scaler)
                ])),
                ("unusualword", Pipeline([
                    ("extract", UnusualWord()),
                    ("log_scaling", log_scaler)
                ])),
                ("coherence", Coherence())
            ]))
        ])),
        ("original_text_features", Pipeline([
            ("features", FeatureUnion([
                ("wordcount", Pipeline([
                    ("extract", HowManyWords()),
                    ("scaler", MinMaxScaler())
                ])),
                ("readingease", Pipeline([
                    ("extract", ReadingEase()),
                    ("scaler", MinMaxScaler())
                ])),
                ("gunningfog", Pipeline([
                    ("extract", GunningFog()),
                    ("scaler", MinMaxScaler())
                ]))
            ]))
        ]))
    ]))
])


feature_names = [
    "cons_density", "stress_value", "sentiment_score",
    "redundance", "unusual_words", "coherence",
    "word_count", "reading_ease", "gunning_fog"
]

In [ ]:
pipeline

In [ ]:
X_processed = pipeline.fit_transform(X)
X_processed_df = pd.DataFrame(X_processed, columns=feature_names)

In [ ]:
X_processed_df

## With parallelism (*not working at the moment*)

In [ ]:
# class InputHandler(BaseEstimator, TransformerMixin):
#     def fit(self, X, y=None):
#         return self

#     def transform(self, X):
#         if isinstance(X, str):
#             X = [X]
#         return pd.DataFrame({"text": X})

# class TextPreprocessor(BaseEstimator, TransformerMixin):
#     def fit(self, X, y=None):
#         return self

#     def get_feature_names_out(self, input_features=None):
#         return ["preprocessed"]

#     def transform(self, X):
#         if isinstance(X, pd.DataFrame):
#             X = X["text"]
#         cleaned = X.apply(basic_cleaning)
#         return pd.DataFrame({"preprocessed": cleaned})

# class ConsDensity(BaseEstimator, TransformerMixin):
#     def fit(self, X, y=None):
#         return self

#     def get_feature_names_out(self, input_features=None):
#         return ["cons_density"]

#     def transform(self, X):
#         return X["preprocessed"].apply(cons_density).values.reshape(-1, 1)

# class Stress(BaseEstimator, TransformerMixin):
#     def fit(self, X, y=None):
#         return self

#     def get_feature_names_out(self, input_features=None):
#         return ["stress_value"]

#     def transform(self, X):
#         return X["preprocessed"].apply(get_sentence_stress).values.reshape(-1, 1)

# class Sentiment(BaseEstimator, TransformerMixin):
#     def fit(self, X, y=None):
#         return self

#     def get_feature_names_out(self, input_features=None):
#         return ["sentiment_score"]

#     def transform(self, X):
#         return X["preprocessed"].apply(sentiment_polarity).values.reshape(-1, 1)

# class Redundance(BaseEstimator, TransformerMixin):
#     def fit(self, X, y=None):
#         return self

#     def get_feature_names_out(self, input_features=None):
#         return ["redundance"]

#     def transform(self, X):
#         return X["preprocessed"].apply(redundance).values.reshape(-1, 1)

# class UnusualWord(BaseEstimator, TransformerMixin):
#     def fit(self, X, y=None):
#         return self

#     def get_feature_names_out(self, input_features=None):
#         return ["unusual_words"]

#     def transform(self, X):
#         return X["preprocessed"].apply(word_choice).values.reshape(-1, 1)

# class Coherence(BaseEstimator, TransformerMixin):
#     def fit(self, X, y=None):
#         return self

#     def get_feature_names_out(self, input_features=None):
#         return ["coherence"]

#     def transform(self, X):
#         return X["preprocessed"].apply(coherence).values.reshape(-1, 1)

# class ReadingEase(BaseEstimator, TransformerMixin):
#     def fit(self, X, y=None):
#         return self

#     def get_feature_names_out(self, input_features=None):
#         return ["reading_ease"]

#     def transform(self, X):
#         return X["text"].apply(reading_ease).values.reshape(-1, 1)

# class GunningFog(BaseEstimator, TransformerMixin):
#     def fit(self, X, y=None):
#         return self

#     def get_feature_names_out(self, input_features=None):
#         return ["gunning_fog"]

#     def transform(self, X):
#         return X["text"].apply(gunning_fog).values.reshape(-1, 1)


In [ ]:
# pipeline = Pipeline([
#     ("input_handler", InputHandler()),
#     ("union", FeatureUnion([
#          ("preprocessed_features", Pipeline([
#             ("preprocessor", TextPreprocessor()),
#             ("features", FeatureUnion([
#                 ("cons_density", ConsDensity()),
#                 ("stress_value", Stress()),
#                 ("sentiment_score", Sentiment()),
#                 ("redundance", Redundance()),
#                 ("unusualword", UnusualWord()),
#                 ("coherence", Coherence())
#             ]))
#         ])),
#         ("original_text_features", FeatureUnion([
#             ("readingease", ReadingEase()),
#             ("gunningfog", GunningFog())
#         ]))
#     ], n_jobs=-1))
# ])


# feature_names = [
#     "cons_density", "stress_value", "sentiment_score",
#     "redundance", "unusual_words", "coherence",
#     "reading_ease", "gunning_fog"
# ]

In [ ]:
# pipeline

In [ ]:
# X

In [ ]:
# X_processed = pipeline.fit_transform(X)
# X_processed_df = pd.DataFrame(X_processed, columns=feature_names)

In [ ]:
# processed_df.shape

In [ ]:
# X_processed_df = pd.DataFrame(X_processed)
# X_processed_df.head()

In [ ]:
# X_processed_df.shape

In [ ]:
# X_final = pd.concat([X, X_processed_df], axis=1)
# X_final.shape

# Model Pipeline only

In [ ]:
import pandas as pd
data_load = pd.read_csv("7k_sampled_dataset.csv")
data = data_load.copy()
data.head()

In [ ]:
data["AI_gen"] = data["source"].apply(lambda x: 0 if x == "Human" else 1)

In [ ]:
X=pd.DataFrame(data["text"])
y=data["AI_gen"]

In [ ]:
import joblib
from roma_pipeline import InputHandler, HowManyWords, TextPreprocessor, ConsDensity, Stress, Sentiment,Redundance,UnusualWord,Coherence,ReadingEase,GunningFog, log_transform
pipeline = joblib.load("roma_pipeline.joblib")

In [ ]:
X_processed_df = pipeline.fit_transform(X)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train,y_test = train_test_split(X_processed_df,y,train_size=0.7, random_state= 1, stratify= y)
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

## Deep Learning

In [ ]:
# pip install tensorflow

In [ ]:
from tensorflow.keras import models, layers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import optimizers
from tensorflow.keras.metrics import Recall


In [ ]:
def initialize_model():
    model = models.Sequential()

    model.add(layers.Dense(256, activation="relu", input_dim=X_processed_df.shape[1]))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.3))


    model.add(layers.Dense(128, activation="relu"))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.3))


    model.add(layers.Dense(64, activation="relu"))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.3))


    model.add(layers.Dense(1, activation="sigmoid"))

    return model

In [ ]:
model = initialize_model()

In [ ]:
model.summary()

In [ ]:
def compile_model(model):

    return  model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.001), metrics=['accuracy', Recall()])

In [ ]:
es = EarlyStopping(patience = 20,restore_best_weights=True, monitor='val_loss')
lr_scheduler = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, verbose=1)

compile_model(model)

history = model.fit(
    X_train, y_train,
    epochs=500,
    callbacks = [es, lr_scheduler],
    validation_split = 0.2
    )

In [ ]:
baseline = model.evaluate(X_test,y_test)

In [ ]:
precision = baseline[1]
recall = baseline[2]
print(precision)
print(recall)

In [ ]:
from tensorflow.keras.models import save_model
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import TFSMLayer

In [ ]:
# model.save(filepath=r"home\romaric\code\Romaric1209\GAIDI\notebooks\roma_models\baseline_model.keras")
# model = load_model(r'/home/romaric/code/Romaric1209/GAIDI/notebooks/roma_models/baseline_model.keras')

## KNN Classifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knn_scores =[]
for i in range(1,100):
    knn = KNeighborsClassifier(n_neighbors=i)

    knn.fit(X_train, y_train)

    knn_score= knn.score(X_test, y_test)

    knn_scores.append(knn_score)

In [ ]:
max(knn_scores)

## Decision Trees

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
tree_clf = RandomForestClassifier(criterion='gini',max_depth=3, random_state=0)

In [ ]:
tree_clf.fit(X_train, y_train)

In [ ]:
tree_clf.score(X_test,y_test)

In [ ]:
tree_clf_scores=[]

for i in range(1,100):
    tree_clf = RandomForestClassifier(criterion='entropy',max_depth=i, random_state=0)
    tree_clf.fit(X_train, y_train)
    tree_clf_score= tree_clf.score(X_test,y_test)
    tree_clf_scores.append(tree_clf_score)

max(tree_clf_scores)


### AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import cross_validate

tree_clf_adb = AdaBoostClassifier(
    RandomForestClassifier(criterion='gini',max_depth=3, random_state=0))

cv_results = cross_validate(tree_clf_adb, X_train, y_train, scoring = ["accuracy","roc_auc","f1","recall"], cv=5)

In [ ]:
cv_results

In [ ]:
print("accuracy: " + str(cv_results["test_accuracy"].mean()))
print("ROC-AUC: " + str(cv_results["test_roc_auc"].mean()))
print("F1: " + str(cv_results["test_f1"].mean()))
print("Recall: " + str(cv_results["test_recall"].mean()))

### Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
tree_gb_clf = GradientBoostingClassifier(
   n_estimators=100,
   learning_rate=0.01,
   max_depth=3
)

In [ ]:
tree_gb_clf.fit(X_train,y_train)

In [ ]:
tree_gb_clf.score(X_test,y_test)

## Stacking

In [ ]:
from sklearn.ensemble import StackingClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

In [ ]:
base_models = [
    ("rf", RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)),  # Tuned RF
    ("knn", KNeighborsClassifier(n_neighbors=5)),  # Optimized KNN
    ("tree_gb", GradientBoostingClassifier(learning_rate=0.01, max_depth=3))
]

In [ ]:
meta_model = LogisticRegression()

In [ ]:
stacking_clf = StackingClassifier(
    estimators=base_models,
    final_estimator=meta_model
)

In [ ]:
param_grid = {
    'final_estimator__C': [0.1, 1, 10],  # Regularization strength for Logistic Regression
    'final_estimator__solver': ['liblinear', 'lbfgs']
}

In [ ]:
grid_search = GridSearchCV(stacking_clf, param_grid, cv=5, scoring='recall', n_jobs=-1)

In [ ]:
grid_search.fit(X_train, y_train)

In [ ]:
best_stacking_model = grid_search.best_estimator_

In [ ]:
y_pred = best_stacking_model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report,recall_score
accuracy_score = accuracy_score(y_test, y_pred)
recall_score = recall_score(y_test, y_pred)
accuracy_score
recall_score

In [ ]:
print(classification_report(y_test, y_pred))

# Test

In [ ]:
import joblib
from roma_pipeline import InputHandler, HowManyWords, TextPreprocessor, ConsDensity, Stress, Sentiment,Redundance,UnusualWord,Coherence,ReadingEase,GunningFog, log_transform
pipeline = joblib.load("roma_pipeline.joblib")

In [ ]:
X=input()

In [ ]:
X_new_processed = pipeline.fit_transform(X)

In [ ]:
X_new_processed

In [ ]:
from tensorflow.keras.models import load_model
model = load_model('/home/romaric/code/Romaric1209/GAIDI/notebooks/roma_models/baseline_model.keras')

In [ ]:
model.predict(X_new_processed)

# Model  No Pipeline + TFIDF

In [73]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [74]:
import pandas as pd
data_load = pd.read_csv("5k_sampled_dataset.csv")
data = data_load.copy()
data.head()

,text,source,prompt_id,text_length,word_count
0,... invention ~~for~~ of the last centuries. ~...,Human,0,1524,229
1,Self-Defense Techniques in Martial Arts Essay\...,Human,0,3564,597
2,The impact of robotics on the future of work i...,GPT-3.5,2874,3858,615
3,Taller individuals will likely need a longer t...,Flan-T5-XXL,0,462,80
4,"The Face on Mars, captured by NASA's Viking 1 ...",Falcon-180B,7,2078,347


In [75]:
data["AI_gen"] = data["source"].apply(lambda x: 0 if x == "Human" else 1)

In [76]:
tfidf = TfidfVectorizer(max_features=7000)
X = tfidf.fit_transform(data["text"]).toarray()
y = data["AI_gen"]

In [77]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train,y_test = train_test_split(X,y,train_size=0.7, random_state= 1, stratify= y)
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

(3500, 7000) (1500, 7000) (3500,) (1500,)


## Deep Learning

In [ ]:
# pip install tensorflow

In [78]:
from tensorflow.keras import models, layers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import optimizers
from tensorflow.keras.metrics import Recall

In [79]:
def initialize_model():
    model = models.Sequential()

    model.add(layers.Dense(256, activation="relu", input_dim=X.shape[1]))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.3))


    model.add(layers.Dense(128, activation="relu"))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.3))


    model.add(layers.Dense(64, activation="relu"))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.3))


    model.add(layers.Dense(1, activation="sigmoid"))

    return model

In [80]:
model = initialize_model()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [81]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_8 (Dense)                      │ (None, 256)                 │       1,792,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_6                │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_7                │ (None, 128)                 │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_8                │ (None, 64)                  │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,835,265 (7.00 MB)

 Trainable params: 1,834,369 (7.00 MB)

 Non-trainable params: 896 (3.50 KB)

In [82]:
def compile_model(model):

    return  model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.001), metrics=['accuracy', Recall()])

In [83]:
es = EarlyStopping(patience = 20,restore_best_weights=True, monitor='val_loss')
lr_scheduler = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, verbose=1)

compile_model(model)

history = model.fit(
    X_train, y_train,
    epochs=500,
    callbacks = [es, lr_scheduler],
    validation_split = 0.2
    )

Epoch 1/500
88/88 ━━━━━━━━━━━━━━━━━━━━ 11s 51ms/step - accuracy: 0.7357 - loss: 0.5604 - recall_5: 0.7734 - val_accuracy: 0.5729 - val_loss: 0.6890 - val_recall_5: 0.0000e+00 - learning_rate: 0.0010
Epoch 2/500
88/88 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9474 - loss: 0.1463 - recall_5: 0.9653 - val_accuracy: 0.5729 - val_loss: 0.7536 - val_recall_5: 0.0000e+00 - learning_rate: 0.0010
Epoch 3/500
88/88 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9784 - loss: 0.0719 - recall_5: 0.9745 - val_accuracy: 0.6014 - val_loss: 0.6090 - val_recall_5: 0.0669 - learning_rate: 0.0010
Epoch 4/500
88/88 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9920 - loss: 0.0403 - recall_5: 0.9893 - val_accuracy: 0.8014 - val_loss: 0.3791 - val_recall_5: 0.5987 - learning_rate: 0.0010
Epoch 5/500
88/88 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9938 - loss: 0.0303 - recall_5: 0.9955 - val_accuracy: 0.8843 - val_loss: 0.2882 - val_recall_5: 0.8495 - learning_rate: 0.0010
Epoch 6/500
88/88 ━━━━━━━

In [103]:
baseline = model.evaluate(X_test,y_test)

47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8939 - loss: 0.2630 - recall_5: 0.8370


In [102]:
accuracy = baseline[1]
recall = baseline[2]
print(accuracy)
print(recall)
print(baseline[0])

0.8920000195503235
0.825563907623291
0.2679789662361145


In [86]:
from tensorflow.keras.models import save_model
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import TFSMLayer

In [ ]:
# model.save(filepath=r"home\romaric\code\Romaric1209\GAIDI\notebooks\roma_models\baseline_model.keras")
# model = load_model(r'/home/romaric/code/Romaric1209/GAIDI/notebooks/roma_models/baseline_model.keras')

## KNN Classifier

In [87]:
from sklearn.neighbors import KNeighborsClassifier

In [88]:
knn_scores =[]
for i in range(1,100):
    knn = KNeighborsClassifier(n_neighbors=i)

    knn.fit(X_train, y_train)

    knn_score= knn.score(X_test, y_test)

    knn_scores.append(knn_score)

In [89]:
max(knn_scores)

0.7746666666666666

## Decision Trees

In [90]:
from sklearn.ensemble import RandomForestClassifier

In [91]:
tree_clf = RandomForestClassifier(criterion='gini',max_depth=3, random_state=0)

In [92]:
tree_clf.fit(X_train, y_train)

RandomForestClassifier(max_depth=3, random_state=0)

In [93]:
tree_clf.score(X_test,y_test)

0.8053333333333333

In [94]:
tree_clf_scores=[]

for i in range(1,100):
    tree_clf = RandomForestClassifier(criterion='entropy',max_depth=i, random_state=0)
    tree_clf.fit(X_train, y_train)
    tree_clf_score= tree_clf.score(X_test,y_test)
    tree_clf_scores.append(tree_clf_score)

max(tree_clf_scores)


0.9226666666666666

### AdaBoost

In [95]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import cross_validate

tree_clf_adb = AdaBoostClassifier(
    RandomForestClassifier(criterion='gini',max_depth=3, random_state=0))

cv_results = cross_validate(tree_clf_adb, X_train, y_train, scoring = ["accuracy","roc_auc","f1","recall"], cv=5)

In [96]:
cv_results

{'fit_time': array([49.94783306, 51.40147328, 50.47430325, 49.37428403, 49.56983757]),
 'score_time': array([1.90009427, 1.66344523, 1.62662315, 1.64483118, 1.4080584 ]),
 'test_accuracy': array([0.91428571, 0.92142857, 0.91714286, 0.89714286, 0.88714286]),
 'test_roc_auc': array([0.96315063, 0.96082713, 0.95999586, 0.95930521, 0.94602564]),
 'test_f1': array([0.90963855, 0.91283677, 0.90993789, 0.8902439 , 0.88226528]),
 'test_recall': array([0.97106109, 0.92903226, 0.94516129, 0.94193548, 0.95483871])}

In [97]:
print("accuracy: " + str(cv_results["test_accuracy"].mean()))
print("ROC-AUC: " + str(cv_results["test_roc_auc"].mean()))
print("F1: " + str(cv_results["test_f1"].mean()))
print("Recall: " + str(cv_results["test_recall"].mean()))

accuracy: 0.9074285714285715
ROC-AUC: 0.9578608951203865
F1: 0.9009844775197997
Recall: 0.9484057670366145


### Gradient Boosting

In [98]:
from sklearn.ensemble import GradientBoostingClassifier

In [99]:
tree_gb_clf = GradientBoostingClassifier(
   n_estimators=100,
   learning_rate=0.01,
   max_depth=3
)

In [100]:
tree_gb_clf.fit(X_train,y_train)

GradientBoostingClassifier(learning_rate=0.01)

In [101]:
tree_gb_clf.score(X_test,y_test)

0.872

## Stacking

In [ ]:
from sklearn.ensemble import StackingClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

In [ ]:
base_models = [
    ("rf", RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)),
    ("knn", KNeighborsClassifier(n_neighbors=5)),
    ("tree_gb", GradientBoostingClassifier(learning_rate=0.01, max_depth=3))
]

In [ ]:
meta_model = LogisticRegression()

In [ ]:
stacking_clf = StackingClassifier(
    estimators=base_models,
    final_estimator=meta_model
)

In [ ]:
param_grid = {
    'final_estimator__C': [0.1, 1, 10],  # Regularization strength for Logistic Regression
    'final_estimator__solver': ['liblinear', 'lbfgs']
}

In [ ]:
grid_search = GridSearchCV(stacking_clf, param_grid, cv=5, scoring='recall', n_jobs=-1)

In [ ]:
grid_search.fit(X_train, y_train)

In [ ]:
best_stacking_model = grid_search.best_estimator_

In [ ]:
y_pred = best_stacking_model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report,recall_score
accuracy_score = accuracy_score(y_test, y_pred)
recall_score = recall_score(y_test, y_pred)
print(accuracy_score)
print(recall_score)

In [ ]:
print(classification_report(y_test, y_pred))

# Model Pipeline + Tfidf

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [104]:
import pandas as pd
data_load = pd.read_csv("5k_sampled_dataset.csv")
data = data_load.copy()
data.head()

,text,source,prompt_id,text_length,word_count
0,... invention ~~for~~ of the last centuries. ~...,Human,0,1524,229
1,Self-Defense Techniques in Martial Arts Essay\...,Human,0,3564,597
2,The impact of robotics on the future of work i...,GPT-3.5,2874,3858,615
3,Taller individuals will likely need a longer t...,Flan-T5-XXL,0,462,80
4,"The Face on Mars, captured by NASA's Viking 1 ...",Falcon-180B,7,2078,347


In [5]:
import joblib
from roma_pipeline import InputHandler, HowManyWords, TextPreprocessor, ConsDensity, Stress, Sentiment,Redundance,UnusualWord,Coherence,ReadingEase,GunningFog, log_transform
pipeline = joblib.load("roma_pipeline.joblib")

In [105]:
data["AI_gen"] = data["source"].apply(lambda x: 0 if x == "Human" else 1)

In [106]:
X=pd.DataFrame(data["text"])
y=data["AI_gen"]

In [107]:
X_processed_df = pipeline.fit_transform(X)

In [108]:
tfidf = TfidfVectorizer(max_features=7000)
X_tfidf = tfidf.fit_transform(data["text"]).toarray()


In [18]:
X_processed_df.shape

(5000, 9)

In [19]:
X_tfidf.shape

(5000, 7000)

In [109]:
if isinstance(X_processed_df, np.ndarray):
    X_processed_df = pd.DataFrame(X_processed_df)

X_tfidf_df = pd.DataFrame(X_tfidf)

X_combined = pd.concat([X_processed_df, X_tfidf_df], axis=1)

print(X_combined.shape)


(5000, 7009)


In [49]:
print(X_combined.dtypes)


0       float64
1       float64
2       float64
3       float64
4       float64
         ...   
6995    float64
6996    float64
6997    float64
6998    float64
6999    float64
Length: 7009, dtype: object


In [110]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train,y_test = train_test_split(X_combined,y,train_size=0.7, random_state= 1, stratify= y)
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

(3500, 7009) (1500, 7009) (3500,) (1500,)


## Deep Learning

In [ ]:
# pip install tensorflow

In [32]:
from tensorflow.keras import models, layers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import optimizers
from tensorflow.keras.metrics import Recall

In [39]:
def initialize_model():
    model = models.Sequential()

    model.add(layers.Dense(256, activation="relu", input_dim=X_combined.shape[1]))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.3))


    model.add(layers.Dense(128, activation="relu"))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.3))


    model.add(layers.Dense(64, activation="relu"))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.3))


    model.add(layers.Dense(1, activation="sigmoid"))

    return model

In [40]:
model = initialize_model()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [41]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                      │ (None, 256)                 │       1,794,560 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 128)                 │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_5                │ (None, 64)                  │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,837,569 (7.01 MB)

 Trainable params: 1,836,673 (7.01 MB)

 Non-trainable params: 896 (3.50 KB)

In [36]:
def compile_model(model):

    return  model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.001), metrics=['accuracy', Recall()])

In [53]:
es = EarlyStopping(patience = 20,restore_best_weights=True, monitor='val_loss')
lr_scheduler = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, verbose=1)

compile_model(model)

history = model.fit(
    X_train, y_train,
    epochs=500,
    callbacks = [es, lr_scheduler],
    validation_split = 0.2
    )

Epoch 1/500
88/88 ━━━━━━━━━━━━━━━━━━━━ 11s 59ms/step - accuracy: 0.7542 - loss: 0.5439 - recall_4: 0.7794 - val_accuracy: 0.8043 - val_loss: 0.5957 - val_recall_4: 0.6355 - learning_rate: 0.0010
Epoch 2/500
88/88 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9494 - loss: 0.1591 - recall_4: 0.9585 - val_accuracy: 0.8857 - val_loss: 0.4808 - val_recall_4: 0.8863 - learning_rate: 0.0010
Epoch 3/500
88/88 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9640 - loss: 0.1090 - recall_4: 0.9709 - val_accuracy: 0.8871 - val_loss: 0.3724 - val_recall_4: 0.9064 - learning_rate: 0.0010
Epoch 4/500
88/88 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9892 - loss: 0.0430 - recall_4: 0.9876 - val_accuracy: 0.8943 - val_loss: 0.3185 - val_recall_4: 0.9431 - learning_rate: 0.0010
Epoch 5/500
88/88 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9933 - loss: 0.0327 - recall_4: 0.9929 - val_accuracy: 0.8943 - val_loss: 0.3477 - val_recall_4: 0.9565 - learning_rate: 0.0010
Epoch 6/500
88/88 ━━━━━━━━━━━━━━━

In [54]:
baseline = model.evaluate(X_test,y_test)

47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9115 - loss: 0.2748 - recall_4: 0.9422


In [57]:
precision = baseline[1]
recall = baseline[2]
print(precision)
print(recall)

0.9240000247955322
0.9473684430122375


In [114]:
from tensorflow.keras.models import save_model
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import TFSMLayer

In [ ]:
# model.save(filepath=r"home\romaric\code\Romaric1209\GAIDI\notebooks\roma_models\baseline_model.keras")
# model = load_model(r'/home/romaric/code/Romaric1209/GAIDI/notebooks/roma_models/baseline_model.keras')

## KNN Classifier

In [58]:
from sklearn.neighbors import KNeighborsClassifier

In [59]:
knn_scores =[]
for i in range(1,100):
    knn = KNeighborsClassifier(n_neighbors=i)

    knn.fit(X_train, y_train)

    knn_score= knn.score(X_test, y_test)

    knn_scores.append(knn_score)

In [60]:
max(knn_scores)

0.8486666666666667

## Decision Trees

In [61]:
from sklearn.ensemble import RandomForestClassifier

In [62]:
tree_clf = RandomForestClassifier(criterion='gini',max_depth=3, random_state=0)

In [63]:
tree_clf.fit(X_train, y_train)

RandomForestClassifier(max_depth=3, random_state=0)

In [64]:
tree_clf.score(X_test,y_test)

0.8246666666666667

In [65]:
tree_clf_scores=[]

for i in range(1,100):
    tree_clf = RandomForestClassifier(criterion='entropy',max_depth=i, random_state=0)
    tree_clf.fit(X_train, y_train)
    tree_clf_score= tree_clf.score(X_test,y_test)
    tree_clf_scores.append(tree_clf_score)

max(tree_clf_scores)


0.928

### AdaBoost

In [111]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import cross_validate

tree_clf_adb = AdaBoostClassifier(
    RandomForestClassifier(criterion='gini',max_depth=3, random_state=0))

cv_results = cross_validate(tree_clf_adb, X_train, y_train, scoring = ["accuracy","roc_auc","f1","recall"], cv=5)

In [112]:
cv_results

{'fit_time': array([32.77509546, 31.3852663 , 32.22392654, 33.27716947, 39.65588903]),
 'score_time': array([1.89977717, 1.38122416, 1.3820591 , 1.52064824, 1.54599977]),
 'test_accuracy': array([0.91285714, 0.92      , 0.91285714, 0.91571429, 0.89714286]),
 'test_roc_auc': array([0.95600889, 0.96480562, 0.96089744, 0.96046319, 0.95434243]),
 'test_f1': array([0.90854573, 0.91082803, 0.90658499, 0.90880989, 0.89253731]),
 'test_recall': array([0.97427653, 0.92258065, 0.95483871, 0.9483871 , 0.96451613])}

In [113]:
print("accuracy: " + str(cv_results["test_accuracy"].mean()))
print("ROC-AUC: " + str(cv_results["test_roc_auc"].mean()))
print("F1: " + str(cv_results["test_f1"].mean()))
print("Recall: " + str(cv_results["test_recall"].mean()))

accuracy: 0.9117142857142856
ROC-AUC: 0.9593035157938224
F1: 0.9054611900396706
Recall: 0.9529198215952702


In [116]:
# joblib.dump(tree_clf_adb, "adaboost_model_text.joblib")
# adaboost_model_text = joblib.load("adaboost_model.joblib")

['adaboost_model_text.joblib']

### Gradient Boosting

In [69]:
from sklearn.ensemble import GradientBoostingClassifier

In [70]:
tree_gb_clf = GradientBoostingClassifier(
   n_estimators=100,
   learning_rate=0.01,
   max_depth=3
)

In [71]:
tree_gb_clf.fit(X_train,y_train)

GradientBoostingClassifier(learning_rate=0.01)

In [72]:
tree_gb_clf.score(X_test,y_test)

0.8586666666666667

## Stacking

In [ ]:
from sklearn.ensemble import StackingClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

In [ ]:
base_models = [
    ("rf", RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)),  # Tuned RF
    ("knn", KNeighborsClassifier(n_neighbors=5)),  # Optimized KNN
    ("tree_gb", GradientBoostingClassifier(learning_rate=0.01, max_depth=3))
]

In [ ]:
meta_model = LogisticRegression()

In [ ]:
stacking_clf = StackingClassifier(
    estimators=base_models,
    final_estimator=meta_model
)

In [ ]:
param_grid = {
    'final_estimator__C': [0.1, 1, 10],  # Regularization strength for Logistic Regression
    'final_estimator__solver': ['liblinear', 'lbfgs']
}

In [ ]:
grid_search = GridSearchCV(stacking_clf, param_grid, cv=5, scoring='recall', n_jobs=-1)

In [ ]:
grid_search.fit(X_train, y_train)

In [ ]:
best_stacking_model = grid_search.best_estimator_

In [ ]:
y_pred = best_stacking_model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report,recall_score
accuracy_score = accuracy_score(y_test, y_pred)
recall_score = recall_score(y_test, y_pred)
accuracy_score
recall_score

In [ ]:
print(classification_report(y_test, y_pred))